In [123]:
import pandas as pd
import json
import numpy as np
import os
import glob
import re
import urllib
from bs4 import BeautifulSoup
from pathlib import Path


In [2]:
def ig_unknown(file, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [ ]:
def ig_isotope(measured_df, mat_list, hl_min, hl_max, name):
    precursores = mat_list.split(',')
    isotopes = []
    with open("database/irr_products.json") as f:
        irradiation_products = json.load(f)
    
    for precursor in precursores:
        try:
            isotopes += irradiation_products[precursor]
        
        except:
            irradiation_products[precursor] = load_irr_prod(precursor)
            with open("database/irr_products.json", 'w') as json_out:
                json.dump(irradiation_products, json_out)

    # get unique only
    isotopes = list(set(isotopes))

    db_parameters = ["half-life", "sigm_half-life", "lambda", "sigm_lambda", "isotope", "Ig", "sigm_Ig", "E_tab", "sigm_E_tab"]

    measured_df[[db_parameters]] = np.nan*np.ones(shape=(measured_df.shape[0], len(db_parameters)))

    for isotope in isotopes:
        measured_df = add_db_info(measured_df, isotope)
    
    measured_df.to_csv(f"out/{name}wDBinfo.csv")

In [ ]:
def ig_nat_element(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [ ]:
def ig_element_spec(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [ ]:
def ig_material(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [76]:
#def get_Igamma(file, material, hl_min, hl_max):
#    df = pd.read_csv(file,header=0,index_col=0)
#    name = file.split('.')[0]
#    mat_type, mat_list = material.split(',', maxindex=1)
#    switcher = {
#        0: ig_unknown(df, hl_min, hl_max, name),
#        "isotope": ig_isotope(df, mat_list, hl_min, hl_max, name), 
#        "element_natural": ig_nat_element(df, mat_list, hl_min, hl_max, name),
#        "element_spec": ig_spec_element(df, mat_list, hl_min, hl_max, name),
#        "material": ig_material(df, mat_list, hl_min, hl_max, name)
#    }
#    switcher.get(mat_type, "Invalid material given")

In [114]:
def append_Igamma_dir(parsed_dir):
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")

    Path("./reports_Ig").mkdir(parents=True, exist_ok=True)
    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        append_Igamma(parsed_file)

In [115]:
def append_Igamma(file):
    print(file)
    A, element = re.split(r'(\d+)(\w+)', file)[-3:-1]
    print(A)
    print(element)

    Path("./ig_db").mkdir(parents=True, exist_ok=True)
    if os.path.isfile(f"ig_db/{A}{element}.csv"):
        #ig_df = pd.read_csv(f"ig_db/{ig_file_name}", header=None, index_col=None)
        print("Reading the csv with gammas.")
    else:
        Path("./downloads").mkdir(parents=True, exist_ok=True)
        if os.path.isfile(f"downloads/{A}{element}.html"):
            extract_Igamma(A, element)
            #ig_df = pd.read_csv(f"ig_db/{ig_file_name}", header=None, index_col=None)
            print("Reading the csv with gammas.")
        else:
            download(A, element)
            extract_Igamma(A, element)
            #ig_df = pd.read_csv(f"ig_db/{ig_file_name}", header=None, index_col=None)
            print("Reading the csv with gammas.")

In [116]:
def getZ(element):
    element_Z ={
        "U": 92,
        "Pu": 94
    }
    return element_Z[element]

In [117]:
def download(A, element):
    #Path(f'downloads/{A}{element}.html').touch()
    url = f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={getZ(element)}0{A}"
    urllib.request.urlretrieve (url, f"downloads/{A}{element}.html")
    print(f"File {A}{element}.html downloaded.")

In [120]:
def extract_Igamma(A, element):
    html_file = f"downloads/{A}{element}.html"
    soup = BeautifulSoup(html_file.read(), 'lxml')
    
    gammas_table = soup.find_all("table")[4]
    gammas_rows = gammas_table.find_all('tr')[3:-1]
    energy = []
    sigm_energy = []
    i = []
    sigm_i = []

    for row in gammas_rows:
        cells = row.find_all('td')
        
        e_val = float(cells[0].get_text(strip=True))
        i_val = float(cells[1].get_text(strip=True))
        
        energy.append(e_val[:-1])
        sigm_energy.append(e_val[-1])
        i_list.append(i_val[:-1])
        sigm_i.append(i_val[-1])


    
    #file = open(f"downloads/{A}{element}.html", 'r')
    #copy_line = False
    #gamma_lines = []
    #for line in file.readlines():
    #    if "A NAME=\"g\"" in line:
    #        copy_line = True
    #        continue
    #    if copy_line:
    #        gamma_lines.append(line)
    #    
    #    if "A NAME=\"x\"" in line:
    #        break
    #    
    #file.close()

    with open(f"ig_db/{A}{element}.csv", "w") as f:
        for ig in gamma_lines:
            f.write(f"{ig}\n")
        f.close()
    print(f"Ig extracted from file 'downloads/{A}{element}.html' into 'ig_db/{A}{element}.csv'.")

In [214]:
html_doc = open("downloads/238U.html", "r")

soup = BeautifulSoup(html_doc.read(), 'lxml')
gammas_table = soup.find_all("table")[4]
gammas_rows = gammas_table.find_all('tr')[3:-1]
gamma_energy_list = []
gamma_i_list = []

for row in gammas_rows:
    cells = row.find_all('td')
    e_val = cells[0].get_text(strip=True)

    gamma_energy_list.append(e_val)
    gamma_i_list.append(cells[1].get_text(strip=False))


print(gamma_energy_list)
print(gamma_i_list)

#print(html_doc.read())

['49.556', '113.51']
['0.064\xa08\xa0', '0.0102\xa015\xa0']


In [122]:
append_Igamma_dir("parsed_reports")

parsed_reports/238U.csv
238
U
Ig extracted from file 'downloads/238U.html' into 'ig_db/238U.csv'.
Reading the csv with gammas.
parsed_reports/239Pu.csv
239
Pu
Ig extracted from file 'downloads/239Pu.html' into 'ig_db/239Pu.csv'.
Reading the csv with gammas.
